In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [3]:
import acquire_zillow
import prepare_zillow

In [4]:
df = pd.read_csv('test.csv')

/usr/local/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (63) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df.shape

(167854, 68)

In [6]:
df = prepare_zillow.prep_zillow(df)
df = df.drop_duplicates(subset='parcelid')
df.drop('Unnamed: 0',axis=1,inplace = True)

In [7]:
df.shape

(157174, 68)

### Will be replaced with proper imputation in prep

In [11]:
df = df.replace(np.nan,0)

In [20]:
df.drop('Unnamed: 0',axis=1,inplace = True)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157174 entries, 0 to 167853
Data columns (total 67 columns):
parcelid                        157174 non-null int64
airconditioningtypeid           157174 non-null float64
architecturalstyletypeid        157174 non-null float64
basementsqft                    157174 non-null float64
bathroomcnt                     157174 non-null float64
bedroomcnt                      157174 non-null float64
buildingclasstypeid             157174 non-null float64
buildingqualitytypeid           157174 non-null float64
calculatedbathnbr               157174 non-null float64
decktypeid                      157174 non-null float64
finishedfloor1squarefeet        157174 non-null float64
calculatedfinishedsquarefeet    157174 non-null float64
finishedsquarefeet12            157174 non-null float64
finishedsquarefeet13            157174 non-null float64
finishedsquarefeet15            157174 non-null float64
finishedsquarefeet50            157174 non-null flo

In [23]:
list(df.columns)

['parcelid',
 'airconditioningtypeid',
 'architecturalstyletypeid',
 'basementsqft',
 'bathroomcnt',
 'bedroomcnt',
 'buildingclasstypeid',
 'buildingqualitytypeid',
 'calculatedbathnbr',
 'decktypeid',
 'finishedfloor1squarefeet',
 'calculatedfinishedsquarefeet',
 'finishedsquarefeet12',
 'finishedsquarefeet13',
 'finishedsquarefeet15',
 'finishedsquarefeet50',
 'finishedsquarefeet6',
 'fips',
 'fireplacecnt',
 'fullbathcnt',
 'garagecarcnt',
 'garagetotalsqft',
 'hashottuborspa',
 'heatingorsystemtypeid',
 'latitude',
 'longitude',
 'lotsizesquarefeet',
 'poolcnt',
 'poolsizesum',
 'pooltypeid10',
 'pooltypeid2',
 'pooltypeid7',
 'propertycountylandusecode',
 'propertylandusetypeid',
 'propertyzoningdesc',
 'rawcensustractandblock',
 'regionidcity',
 'regionidcounty',
 'regionidneighborhood',
 'regionidzip',
 'roomcnt',
 'storytypeid',
 'threequarterbathnbr',
 'typeconstructiontypeid',
 'unitcnt',
 'yardbuildingsqft17',
 'yardbuildingsqft26',
 'yearbuilt',
 'numberofstories',
 'firep

In [25]:
dfo = df.copy()

In [26]:
df = df[['parcelid',
 'airconditioningtypeid',
 'architecturalstyletypeid',
 'basementsqft',
 'bathroomcnt',
 'bedroomcnt',
 'buildingclasstypeid',
 'buildingqualitytypeid',
 'calculatedbathnbr',
 'decktypeid',
 'finishedfloor1squarefeet',
 'calculatedfinishedsquarefeet',
 'fips',
 'fireplacecnt',
 'fullbathcnt',
 'garagetotalsqft',
 'latitude',
 'longitude',
 'lotsizesquarefeet',
 'logerror']]

In [ ]:
sns.pairplot(df)